### In this notebook, I first generate faces using captions from COCO dataset using ERNIE-ViLG, and then, I using face detector to extract faces from the images generated. 

In [ ]:
#!pip install translators --upgrade

Load caption data

In [ ]:
import paddlehub as hub
import translators as ts
lines = open('face_captions.txt').read().splitlines()

In [ ]:
import random
random.seed(42)
#Generate 1000 random numbers between 0 and 9999
# I plan to generate about 600 images, but may need to come back generate more if some of the images do not contain a good face
randomlist = random.sample(range(0, 9999), 1100)

In [ ]:
len(randomlist)

In [ ]:
randomlist[:10]

Start generating face images

In [ ]:
import paddlehub as hub
import translators as ts
module = hub.Module(name="ernie_vilg")
for i in range(500, 600):
    prompt = ts.baidu(lines[randomlist[i]] + 'showing front face, like a photo in real life', professional_field='common')
    try: # for certain prompts, error message "暂不支持创作该内容，请修改后再试 (does not support this prompt, please modify)" was returned
        module.generate_image(text_prompts=prompt, style='写实风格',  output_dir='./generated_face_images_with_ERNIE-ViLG_version_2/',topk=1)
        print(i)      
    except:
        continue

In [ ]:
i

extract faces from the generated face images 

In [ ]:
#set up folders for extracted faces 
# initialize COCO API for instance annotations
import os
dataDir = './'

image_folder = "extracted_generated_faces_version2"

savedirimage = dataDir + image_folder



if not os.path.exists(savedirimage):
	os.makedirs(savedirimage)

In [ ]:
# install all packages
!pip install keras
!pip install tensorflow
from keras.models import load_model
!pip install mtcnn
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

import numpy as np
from mtcnn.mtcnn import MTCNN
from numpy import asarray
from PIL import Image
import matplotlib.pyplot as plt


# Simple python package to shut up Tensorflow warnings and logs.
!pip install silence_tensorflow
import silence_tensorflow.auto

I did not use threshold as filtering criteria because this will gaurantee low FID score later. I still kept the dimension requirement. 

In [ ]:
# extract a single face from a given photograph
def extract_face(filename, index, required_size=(250, 250)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    if len(results) >=1:
        x1, y1, width, height = results[0]['box']
        if height-width>=15:
            # bug fix
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            # extract the face
            face = pixels[y1:y2, x1:x2]
            # resize pixels to the model size
            image = Image.fromarray(face)
            image = image.resize(required_size)
            face_array = np.asarray(image)
            plt.imsave(os.path.join(savedirimage, 'face_img_{}.jpg'.format(index)),face_array)
            return True
    return False

In [ ]:
files = os.listdir('generated_face_images_with_ERNIE-ViLG_version_2/')

In [ ]:
len(files)

In [ ]:
index =0
for file in files:
    try:
        if extract_face(os.path.join('./generated_face_images_with_ERNIE-ViLG_version_2/',file), index = index):
            print(index)
            index+=1
    except:
        None

Combine to one folder

In [ ]:
img_path = 'filtered_generated_face/'
files = os.listdir('filtered_generated_face/')

import os
dataDir = './'

image_folder = 'combined_filtered_generated_faces'

savedirimage = dataDir + image_folder

if not os.path.exists(savedirimage):
    os.makedirs(savedirimage)

In [ ]:
import cv2
count = 495
for f_name in files:
    image = cv2.imread(os.path.join(img_path,f_name))
    cv2.imwrite(os.path.join(savedirimage, 'face_img_{}.jpg'.format(count)), image)
    count+=1

In [ ]:
count

resize to 100x100

In [1]:
import os
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from PIL import Image
img_path = '../extracted_faces_from_COCO/'
files = os.listdir('../extracted_faces_from_COCO/')

dataDir = './'

image_folder = 'extracted_face_from_COCO_100_by_100/'

savedirimage = dataDir + image_folder

if not os.path.exists(savedirimage):
    os.makedirs(savedirimage)

In [2]:
len(files)

10000

In [3]:
import cv2
count = 0
for f_name in files:
    #print(f_name)
    image = Image.open(os.path.join(img_path,f_name))
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    image = Image.fromarray(pixels)
    image = image.resize((100,100))
    face_array = np.asarray(image)
    plt.imsave(os.path.join(savedirimage, 'face_img_{}.jpg'.format(count)),face_array)
    count+=1

In [14]:
image = cv2.imread(os.path.join('combined_filtered_generated_faces_100_by_100/','face_img_0.jpg'))

Testing out [mediaPipe](https://google.github.io/mediapipe/solutions/face_detection.html)

In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# For static images:
IMAGE_FILES = ['ERNIE_ViLG_Good_face_4.png']
with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
    for idx, file in enumerate(IMAGE_FILES):
        image = cv2.imread(file)
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Draw face detections of each face.
        if not results.detections:
            continue
        annotated_image = image.copy()
        for detection in results.detections:
            print('Nose tip:')
            print(mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
            mp_drawing.draw_detection(annotated_image, detection)
        cv2.imwrite('annotated_image' + str(idx) + '.png', annotated_image)

In [ ]:
results

In [ ]:
results.detections
